In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import cv2
import math
import numpy as np
from skimage import measure
from skimage.measure import regionprops
from skimage.morphology import opening, erosion, dilation, disk
import matplotlib.pyplot as plt

In [0]:
PRED_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Ki67/KFold LadderNet/"
GT_PATH  = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Ki67/Masks/"

In [0]:
FOLD_PATH = []
files = sorted(os.listdir(PRED_PATH))

# Removing all the .np files from list of folds
for i in range(len(files)):
  if(files[i].split('.')[-1]=="npy"):
    continue
  FOLD_PATH.append(files[i])
print(FOLD_PATH)

['Fold 0', 'Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5', 'Fold 6', 'Fold 7', 'Fold 8']


In [0]:
def updateConfusionMatrix(ConfusionMatrix, PredictedImage, GroundTruth, axis):
    # Opening operation
    selem = disk(15)
    GroundTruth = dilation(GroundTruth,selem)
    GroundTruthLabel = measure.label(GroundTruth)

    rows,cols = PredictedImage.shape

    #  0 is background
    for label in range(1, GroundTruthLabel.max()+1):
      masklabel = np.zeros([rows,cols])
      masklabel[GroundTruthLabel==label] = 1

      resultantimage = cv2.multiply(PredictedImage, masklabel)

      areaGT = np.sum(masklabel)
      areaSM = np.sum(resultantimage)

      if(areaSM > 0 ):
        ConfusionMatrix[axis][0] = ConfusionMatrix[axis][0] + 1       #TP
        PredictedImage[GroundTruthLabel==label] = 0
      else:
        ConfusionMatrix[axis][2] = ConfusionMatrix[axis][2] + 1       #FN
    
    predictedprops = regionprops(measure.label(PredictedImage))

    for i in range(len(predictedprops)):
      if(predictedprops[i].area >= 2000):
        ConfusionMatrix[axis][1] += 1                                #FP

In [0]:
def getPatientWiseCount(ConfusionMatrix, PredictedPatient, GroundTruthPatient):

  for imagenum in range(PredictedPatient.shape[0]):
  
    PredictedImage = PredictedPatient[imagenum]
    GroundTruth    = GroundTruthPatient[imagenum]
    rows,cols = PredictedImage.shape

    # dividing the image into strong,weak,intermediate
    predictedIP = np.zeros([rows,cols])
    predictedIN   = np.zeros([rows,cols])
    
    gtIP = np.zeros([rows,cols])
    gtIN = np.zeros([rows,cols])
    
    predictedIP[PredictedImage==255] = 1
    predictedIN[PredictedImage==128] = 1

    gtIP[GroundTruth==255] = 1
    gtIN[GroundTruth==128] = 1
    
    # 0th axis in confusion matrix is immunopositive
    # 1st axis in confusion matrix is immunonegative

    updateConfusionMatrix(ConfusionMatrix, predictedIP, gtIP,  0)
    updateConfusionMatrix(ConfusionMatrix, predictedIN, gtIN,  1)

In [0]:
def getMetrics(m, axis):
  precision = 1
  recall = 1
  dice = 1
  if(m[axis][0] + m[axis][1] != 0):
    precision = m[axis][0] / (m[axis][0] + m[axis][1])

  if(m[axis][0] + m[axis][2] != 0):
    recall    = m[axis][0] / (m[axis][0] + m[axis][2])
    
  if(precision + recall != 0):
    dice      = (2 * precision * recall) / (precision + recall)
  else:
    dice = 0

  return [precision, recall, dice]

In [0]:
# Axis 0 = Fold
# Axis 1 = Class (Immunopositive Immunonegative)
# Axis 2 = Metric (Precision Recall Dice)
objEvaluationTrain = np.empty((0,2,3))
objEvaluationTest = np.empty((0,2,3))
score = np.zeros((1,2,3))

In [0]:
for fold in range(len(FOLD_PATH)):

  print("Fold: " + str(fold))
  
  train_path = os.listdir(PRED_PATH + FOLD_PATH[fold] + '/Train/')
  test_path = os.listdir(PRED_PATH + FOLD_PATH[fold] + '/Test/')

  train_confusion_matrix = np.zeros([ 2, 3])
  test_confusion_matrix  = np.zeros([ 2, 3])

  # TRAIN
  for patientID in train_path:    
    SM = np.load(PRED_PATH + FOLD_PATH[fold] + '/Train/' + patientID)
    GT = np.load(GT_PATH + 'IHC ' + patientID)
    getPatientWiseCount(train_confusion_matrix, SM, GT)
    print(train_confusion_matrix)

  for i in range(2):
    metrics = getMetrics(train_confusion_matrix, i)
    for j in range(len(metrics)):
      score[0][i][j] = metrics[j]
    
  objEvaluationTrain = np.append(objEvaluationTrain,score,axis=0)
  print(score)

  # TEST
  for patientID in test_path:
    SM = np.load(PRED_PATH + FOLD_PATH[fold] + '/Test/' + patientID)
    GT = np.load(GT_PATH + 'IHC ' + patientID)
    getPatientWiseCount(test_confusion_matrix, SM, GT)

  for i in range(2):
    metrics = getMetrics(test_confusion_matrix, i)
    for j in range(len(metrics)):
      score[0][i][j] = metrics[j]
  
  objEvaluationTest = np.append(objEvaluationTest,score,axis=0)
  print(score)

Fold: 0
[[626.   9.  27.]
 [427.   2.  19.]]
[[759.  10.  33.]
 [682.  14.  23.]]
[[1031.   16.   33.]
 [1119.   24.   28.]]
[[1247.   53.   48.]
 [1599.   29.   32.]]
[[1598.   55.   52.]
 [1928.   35.   46.]]
[[1920.   57.   62.]
 [2355.   43.   53.]]
[[2072.   58.   66.]
 [2758.   68.   55.]]
[[2309.   62.   67.]
 [3150.   87.   55.]]
[[[0.9738507  0.97180135 0.97282494]
  [0.97312326 0.98283931 0.97795716]]]
[[[0.90625    0.8826087  0.89427313]
  [0.77977162 0.95409182 0.85816876]]]
Fold: 1
[[203.   0.  27.]
 [484.  26.  17.]]
[[328.   1.  41.]
 [726.  30.  34.]]
[[ 594.    3.   47.]
 [1142.   44.   60.]]
[[ 786.   47.   86.]
 [1618.   81.   68.]]
[[1121.   48.  106.]
 [1912.   81.  117.]]
[[1433.   49.  126.]
 [2333.   96.  130.]]
[[1579.   49.  136.]
 [2686.  105.  182.]]
[[1814.   51.  139.]
 [3076.  142.  184.]]
[[[0.97265416 0.92882744 0.95023573]
  [0.95587321 0.94355828 0.94967583]]]
[[[0.97993311 0.89739663 0.93685052]
  [0.94348894 0.86098655 0.9003517 ]]]
Fold: 2
[[218.  

In [0]:
print(objEvaluationTrain.shape)
print(objEvaluationTest.shape)

(9, 2, 3)
(9, 2, 3)


In [0]:
SAVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Ki67/KFold LadderNet/"
np.save(SAVE_PATH + "Object-wise Metrics Train.npy", objEvaluationTrain)
np.save(SAVE_PATH + "Object-wise Metrics Test.npy", objEvaluationTest)

In [0]:
#To calculate average, precision and recall
LOAD_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/Ki67/KFold LadderNet/"
trainMetrics = np.load(LOAD_PATH + "Object-wise Metrics Train.npy")
testMetrics  = np.load(LOAD_PATH + "Object-wise Metrics Test.npy")
print(trainMetrics.shape)
print(testMetrics.shape)


(9, 2, 3)
(9, 2, 3)


In [0]:

#For pixel-wise background is included, object-wise background is not included
numClasses = 2
avgTrainMetrics  = np.zeros([numClasses, 3])
avgTestMetrics   = np.zeros([numClasses, 3])

def calculateMetrics(AvgMetrics, ValidationMetrics):
  for patientNum in range(ValidationMetrics.shape[0]):
    for row in range(numClasses):
      for col in range(3):
        AvgMetrics[row][col] += ValidationMetrics[patientNum][row][col]
  AvgMetrics = np.divide(AvgMetrics,ValidationMetrics.shape[0] )
  return AvgMetrics


avgTrainMetrics = calculateMetrics(avgTrainMetrics, trainMetrics)
print('TRAIN :')
print(avgTrainMetrics)

avgTestMetrics = calculateMetrics(avgTestMetrics, testMetrics)
print('TEST :')
print(avgTestMetrics)




TRAIN
[[0.94444232 0.97314069 0.95785052]
 [0.95061171 0.96484392 0.95674138]]
TEST
[[0.89499355 0.94716618 0.91797318]
 [0.87424072 0.90596679 0.87815688]]


**SM = Segmented Mask**

**GT = Ground Truth** 